In [3]:
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading, SelfTrainingClassifier
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit



# iris = datasets.load_iris()
# rng = np.random.RandomState(42)
# random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
# labels = np.copy(iris.target)
# labels[random_unlabeled_points] = -1
# label_prop_model.fit(iris.data, labels)

In [4]:
import os
# from google.colab import drive
# drive.mount(r'/content/gdrive')
# print(os.listdir())
# os.chdir(r'./gdrive/MyDrive/congress_SSL')
os.chdir(r'D:\projects\congress\\')
os.getcwd()

'D:\\projects\\congress'

In [5]:
congress = pd.read_csv(r'./processed/processed_reports/crs_2009_20_10_04_trancated.csv',  sep ='\t', encoding = 'ISO-8859-1', index_col = 0)

In [6]:
congress.head()

,speech_id,true_id,chamber,state,party,congress,daysafter,monthsafter,speech,date,topics_prior
594060,1110000007,L000557,H,CT,D,111,6578,216,our democracy renews itself every 2 years as m...,20090106,22
594061,1110000009,P000587,H,IN,R,111,6578,216,madam clerk. as chairman of the republican con...,20090106,22
594062,1110000013,B000589,H,OH,R,111,6578,216,. leader hoyer. fellow members. and a special ...,20090106,18
594063,1110000014,P000197,H,CA,D,111,6578,216,thank you very much. leader boehner. together....,20090106,22
594064,1110000016,L000557,H,CT,D,111,6578,216,"for""your nomination this morning. thank you to...",20090106,-1


In [7]:
sent_vec = np.load(r'./processed/training_vectors/sentence_vec_2009_20.npy')
As_all = np.load(r'./processed/training_vectors/author_dummy_2009_20.npy')

In [8]:
sent_vec.shape, congress.shape, As_all.shape

((208292, 300), (208292, 11), (208292, 1674))

In [9]:
dws = np.load(r'./processed/training_vectors/dws_2009_20.npy')
congress['dw']= dws


true_switchers_ = ['A000361','B000229', 'B001264', 'D000168', 'G000280', 'H000067', 'H000390', 'L000119', 'P000066', 'T000058', 'C000077','F000257', 'G000557', 
'S000320', 'S000709','J000072']

independents = ['S000033', 'B001237', 'K000383']

true_switchers =independents + true_switchers_
select = ~congress['true_id'].isin(true_switchers)
congress_flt_ = congress.loc[select]
sentvec_flt = sent_vec[select]
dws_flt = dws[select]
labels = ((congress_flt_['party'] == 'D')*1).values
As_flt =As_all[select]

# congress_flt = congress_flt_.sort_values(by = ['dw'], ascending = True, ignore_index = True)

In [10]:
labels, labels.shape, As_flt.shape

(array([1, 0, 0, ..., 0, 0, 1]), (206899,), (206899, 1674))

In [11]:
# labels.shape, congress_flt.shape
topics = congress_flt_['topics_prior'].values
topics.shape

(206899,)

In [12]:
# sort dws
sortinx = np.argsort(dws_flt)
dws_flt[sortinx]

array([-0.769, -0.769, -0.769, ...,  0.916,  0.916,  0.916], dtype=float32)

In [13]:
labels = labels[sortinx]
sentvec_flt = sentvec_flt[sortinx]
topics = topics[sortinx]
As = As_flt[sortinx]

In [14]:
def sample(labels, topics, biased = True, notmask = 0.8):

  total = len(labels)
  print(total)
  if not biased: 
    print('started random split, test size {}'.format(1-notmask))
    sss = StratifiedShuffleSplit(n_splits=5, test_size= 1- notmask)
    trainid, testid = next(sss.split(topics, labels))
    assert len(trainid) + len(testid) == total, "length error"
  else:
    print('started biased split, test size {}'.format(1-notmask))
    topn = int(total * notmask/2.0)
    testid = np.arange(total)[topn:-topn]
    trainid = np.concatenate([np.arange(total)[:topn], np.arange(total)[-topn:]])
  return trainid, testid

In [23]:
def predict_party_membership(s_pred, y, As):
    # filter out nan
    
    fin = np.isfinite(s_pred)
    s_pred = s_pred[fin]
    y = y[fin]
    As = As[fin]
    nonzero_inx = (np.sum(As.T, axis = -1) !=0)
    ave_slants = (np.matmul(As.T, s_pred)/(np.sum(As.T, axis = -1) + 0.000001))[nonzero_inx]
    y_ = np.matmul(As.T, y) >0 # this is party label per person, but those with 0 speech be 0
    print("test slant", ave_slants.mean(), ave_slants[:5])
    y_true = y_[nonzero_inx]
    y_pred = ave_slants>0
    return np.mean(y_true == y_pred)

In [24]:
trainid, testid = sample(labels, topics, biased = True, notmask =0.2)

206899
started biased split, test size 0.8


In [25]:
del congress
del congress_flt_


NameError: name 'congress' is not defined

In [26]:
del sent_vec
del dws

NameError: name 'sent_vec' is not defined

In [31]:
rounds = 6
topn = 50000000

notmasks = [0.01, 0.03, 0.05, 0.08, 0.2, 0.4, 0.6, 0.8]

# notmasks = [0.01]
# topn = 5000
# rounds = 2
model_name = 'label_spreading'
label_prop_model = LabelSpreading(kernel= 'knn', n_neighbors=10)
for biased in [True, False]:
  bia = 'bias'
  if not biased:
    bia = 'unbias'
  f = open( r'./results/SKSSL/{}_{}.txt'.format(model_name, bia), 'w')
  f.write('\taccs\tpccs\n')
  for notmask in notmasks:
    version = "{}_{}_{}".format(model_name, notmask, bia)
    print(version)
    
    if biased:
#       continue
      trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)

      ys = np.copy(labels)
      ys[testid] = -1

      Xs = np.copy(sentvec_flt)
      Xidx = np.arange(len(Xs))
      
      np.random.shuffle(Xidx)
      Xs = Xs[Xidx]
      ys = ys[Xidx]
      print(ys[:100])
      label_prop_model.fit(Xs[:topn], ys[:topn])
      Xt = sentvec_flt[testid]
      yt = labels[testid]
      At = As[testid]
      y_pred = label_prop_model.predict(Xt)
      acc = np.mean(y_pred == yt)
#       print(acc)
      proba_pred = label_prop_model.predict_proba(Xt)
      s_pred = np.array([np.log(i[1]*1000)- np.log(i[0]*1000) for i in proba_pred])
      print(y_pred.mean(), len(s_pred) - np.isfinite(s_pred).sum(), len(s_pred))

      pcc = predict_party_membership(s_pred, yt, At)


    
      f.write(version)
      f.write('\t'+str(acc))
      print("acc, pcc", acc, pcc)
      f.write('\t' + str(pcc))
      f.write('\n')
    else:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)
        ys = np.copy(labels)
        ys[testid] = -1
        
        Xs = np.copy(sentvec_flt)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        label_prop_model.fit(Xs[:topn], ys[:topn])
        Xt = sentvec_flt[testid]
        yt = labels[testid]
        At = As[testid]
        y_pred = label_prop_model.predict(Xt)
        acc = np.mean(y_pred == yt)
#         print(acc)
        accs.append(acc)
        
        proba_pred = label_prop_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*1000)- np.log(i[0]*1000) for i in proba_pred])
        print(y_pred.mean(), len(s_pred) - np.isfinite(s_pred).sum())
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
      f.write(version)
      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      print("acc, pcc", accs, pccs)
      f.write('\n')
  f.close()

label_spreading_0.01_bias
206899
started biased split, test size 0.99
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log


0.5218985407482266 9308 204831
test slant 0.8683953101542335 [-0.07395268  1.77464752  0.35814688  2.76239974 -0.51464617]
acc, pcc 0.6055186959005229 0.7180746561886051
label_spreading_0.03_bias
206899
started biased split, test size 0.97
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log


0.49696800585969614 7660 200693
test slant 0.6982119385728949 [-0.42931036  0.78204197  0.71002518  1.89594189 -1.11324174]
acc, pcc 0.6009128370197266 0.7134328358208956
label_spreading_0.05_bias
206899
started biased split, test size 0.95
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  0 -1 -1 -1]


...

0.5454809086515224 1025
test slant 0.32200486940270984 [ 2.07427015  0.65172609 -0.48763792  2.00340276  0.40498422]
206899
started random split, test size 0.19999999999999996
[ 1  1  1  0 -1  1  1  1  0  0  0  0  1 -1  1  1  0 -1 -1  0  0  1  1  1
 -1  1 -1  0  0 -1  1  0  1 -1  0 -1 -1  1  1  1 -1  1  0  0  0  1  0  0
 -1 -1 -1  0  1  0  1  1  0  0 -1  1  0  1  1  0  0  1  1  0  1  1  0  1
 -1  0  0  1 -1  0  1  0  1  1  0  0 -1  1  1  0  1  0  1  1  1  0  1  0
 -1  1  0  0]


D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in log


0.5460125664572257 990
test slant 0.3515923373888161 [ 1.30328201  3.52642252 -0.24905064  2.90691179  0.127439  ]
206899
started random split, test size 0.19999999999999996
[ 0  0  0  1  0  0  1 -1  0  1  1  0  0  1 -1  0  1  1  1  0  1  0  0  1
  0  0  1  1  1  1  1 -1  0  1  1  0  1  0  0  0  0  1  1  1  1  0  1  0
  0  1  0  0  0 -1  0 -1  1  0 -1  1 -1  1  0  1 -1 -1  1  1  0 -1  0  1
  1  1  0  1  1  1  1  1  0  1  1 -1 -1  1  0  0  1  0  0  1  0  0 -1  0
 -1  0 -1 -1]


D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in log


0.5431367810536492 953
test slant 0.35847846421373153 [ 0.43410442  3.44458793 -0.50262268  1.36641475  0.31640247]
acc, pcc [0.7423392943450943, 0.7426776220396327, 0.7378927017883036, 0.7396326727887869, 0.7406476558724021, 0.7417351377477042] [0.9240759240759241, 0.903096903096903, 0.9007936507936508, 0.9034825870646767, 0.9072781655034895, 0.8944223107569721]
